In [345]:
import pandas as pd
import os
from datetime import datetime
import numpy as np
import calendar
from pathlib import Path

import requests

import quandl
quandl.ApiConfig.api_key = "ca9bMvozbsfJ41tQECsN"

pd.set_option('max_columns',250)
pd.set_option('max_rows',100)

In [171]:
def file_list(ticker):

    statement_folder = f"../data/timeseries/{ticker}/"
    file_list = []
    count = 0 
    for dirname, _, filenames in os.walk(f"{statement_folder}"):
        for filename in filenames:
            file_list.append(os.path.join(dirname,filename))
            print(f"{count}: {filename}")
            count +=1
    return file_list

0: Condensed Consolidated Financial Statement Details - Property Plant and Equipment Net Details.csv
1: Financial Instruments - Derivative Instruments at Gross Fair Value Details.csv
2: Shareholders Equity - Additional Information Details.csv
3: Revenue Recognition - Deferred Revenue Expected Timing of Realization Percentage Details.csv
4: Benefit Plans - Summary of Share-Based Compensation Expense and the Related Income Tax Benefit Details.csv
5: Financial Instruments - Restricted Cash Details.csv
6: Summary of Significant Accounting Policies - Computation of Basic and Diluted Earnings Per Share Details.csv
7: Leases - Additional Information Details.csv
8: Benefit Plans - Additional Information Details.csv
9: Revenue Recognition - Net Sales Disaggregated by Significant Products and Services Details.csv
10: Financial Instruments - Pre-Tax Gains and Losses of Derivative and Non-Derivative Instruments Designated as Cash Flow and Net Investment Hedges Details.csv
11: Financial Instruments

In [321]:
def retrieve_df(list_num):
    print(file_list[list_num])
    df = pd.read_csv(file_list[list_num],index_col=[0,1])
    return df

In [304]:
def fetch_quandl(ticker):
    df_quandl_tmp = quandl.get_table("ZACKS/FC",ticker=ticker,per_type='Q')
    if 'per_end_date' in df_quandl_tmp:
        df_quandl_tmp.index = df_quandl_tmp['per_end_date'].astype(str)
        del(df_quandl_tmp['per_end_date'])
    return df_quandl_tmp

In [305]:
def convert_df(df):
    df_tmp = df
    df_tmp.columns = [f"{date_col[0:4]}-{date_col[5:7]}-{calendar.monthrange(int(date_col[0:4]),int(date_col[5:7]))[1]}" for date_col in df_tmp.columns]
    df_tmp.index = df_tmp.index.droplevel(0)
    df_tmp = df_tmp.loc[:,df_quandl_tmp.index].T/1000000
    return df_tmp

In [327]:
def add_match_columns(df_tmp,df_quandl_tmp,mapping_dict):
    match_count = 0
    for col in df_tmp.columns:
        for quand_col in df_quandl_tmp.columns:
            if (df_quandl_tmp.loc[:,quand_col] == df_tmp.loc[:,col]).all() and ('___' not in col) and (col[:8] == 'us-gaap_'):
                match_count += 1
                if quand_col not in mapping_dict:
                    mapping_dict[quand_col] = []
            
                mapping_dict[quand_col].append(col)
                mapping_dict[quand_col] = list(np.unique(mapping_dict[quand_col]))
                
    print(f"Matches: {match_count}")
    
    return mapping_dict

In [338]:
#CSCO, DIS,'GE','HD','JNJ','MCD', NKE
ticker_list = ['AAPL','AXP','BA','CAT','CVX','DD','GS','IBM','INTC','JPM','KO','MMM','MRK','MSFT','PFE','PG']


In [339]:
mapping_dict = {}
for ticker in ticker_list:
    statement_folder = f"../data/timeseries/{ticker}/Statement/"
    print(ticker)
    for file in os.listdir(statement_folder):
        print(file)
        df_tmp = pd.read_csv(f'{statement_folder}{file}',index_col=[0,1])
        df_tmp = convert_df(df_tmp)
        df_quandl_tmp = fetch_quandl(ticker)
        if (len(df_tmp) == len(df_quandl_tmp)):
            mapping_dict = add_match_columns(df_tmp,df_quandl_tmp,mapping_dict)

AAPL
CONDENSED CONSOLIDATED STATEMENTS OF SHAREHOLDERS EQUITY Unaudited.csv
CONDENSED CONSOLIDATED BALANCE SHEETS Unaudited Parenthetical.csv
Matches: 2
CONDENSED CONSOLIDATED STATEMENTS OF COMPREHENSIVE INCOME Unaudited.csv
Matches: 7
CONDENSED CONSOLIDATED STATEMENTS OF OPERATIONS Unaudited.csv
Matches: 19
CONDENSED CONSOLIDATED STATEMENTS OF CASH FLOWS Unaudited.csv
Matches: 8
CONDENSED CONSOLIDATED BALANCE SHEETS Unaudited.csv
Matches: 20
AXP
Consolidated Statements of Cash Flows Parenthetical.csv
Matches: 0
Consolidated Statements of Income Parenthetical.csv
Matches: 0
Consolidated Balance Sheets Parenthetical.csv
Matches: 0
Consolidated Statements of Cash Flows.csv
Matches: 0
Consolidated Statements of Comprehensive Income.csv
Matches: 0
Consolidated Statements of Shareholders Equity.csv
Matches: 3
Consolidated Statements of Shareholders Equity Parenthetical.csv
Matches: 0
Consolidated Balance Sheets.csv
Matches: 7
Consolidated Statements of Income.csv
Matches: 0
BA
Condensed Con

KeyError: "None of [Index(['2018-03-31', '2018-06-30', '2018-09-30', '2018-12-31'], dtype='object', name='per_end_date')] are in the [columns]"

In [340]:
mapping_dict

{'comm_shares_out': ['us-gaap_commonstocksharesissued',
  'us-gaap_commonstocksharesoutstanding'],
 'income_aft_tax': ['us-gaap_netincomeloss', 'us-gaap_profitloss'],
 'income_cont_oper': ['us-gaap_incomelossfromcontinuingoperations',
  'us-gaap_netincomeloss',
  'us-gaap_profitloss'],
 'income_bef_exord_acct_change': ['us-gaap_netincomeloss',
  'us-gaap_profitloss'],
 'consol_net_income_loss': ['us-gaap_netincomeloss', 'us-gaap_profitloss'],
 'net_income_parent_comp': ['us-gaap_netincomeloss',
  'us-gaap_netincomelossavailabletocommonstockholdersbasic',
  'us-gaap_profitloss'],
 'net_income_loss_share_holder': ['us-gaap_netincomeloss',
  'us-gaap_netincomelossavailabletocommonstockholdersbasic',
  'us-gaap_profitloss'],
 'norm_aft_tax_income': ['us-gaap_netincomeloss', 'us-gaap_profitloss'],
 'tot_revnu': ['us-gaap_revenuefromcontractwithcustomerexcludingassessedtax',
  'us-gaap_revenues'],
 'cost_good_sold': ['us-gaap_costofgoodsandservicessold',
  'us-gaap_costofrevenue'],
 'gross_p

In [347]:
for statement in ['Income','Cash Flow','Balance Sheet']:
    url = f'https://stockrow.com/api/companies/{ticker}/financials.xlsx?dimension=Q&section={statement}%20Statement&sort=desc'
    r = requests.get(url, allow_redirects=True)
    statement_folder = f'../data/stockrow/{ticker}/'
    Path(statement_folder).mkdir(parents=True, exist_ok=True)
    open(f'{statement_folder}/{statement}.xlsx', 'wb').write(r.content)



In [336]:
print(r.content)

b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x00\x98\x9f\xeb\xf6\x8b\x9e\xa1 \x01\x00\x00\x84\x03\x00\x00\x13\x00\x00\x00[Content_Types].xml\xad\x93\xbbn\xc30\x0cE\x7f\xc5\xd0ZXJ:\x14Ea;C\x1fc\x9b!\xfd\x00U\xa2c!zATR\xe7\xefK\xdbM\x87,q\xd0L\x04E\xdes\xaf\x04\xa8Z\xf5\xce\x16\x07Hh\x82\xaf\xd9\x92/X\x01^\x05m\xfc\xb6f\x9f\x9b\xb7\xf2\x91\xad\x9ajs\x8c\x80\x05\xadz\xacY\x97s|\x12\x02U\x07N"\x0f\x11<M\xda\x90\x9c\xcc\xd4\xa6\xad\x88R\xed\xe4\x16\xc4\xfdb\xf1 T\xf0\x19|.\xf3\xc0`M\xf5\x02\xad\xdc\xdb\\<O\xe7\x03\xbaf2Fk\x94\xcc\x94B\x1c\xbc>\x83\x96\xbf@\x9e\xc0\x8e;\xd8\x99\x88w\xb4\xc0\x8a\xd7\x9e(Sz\x9a"\x133\x1c\xce\x85CO\xba\x0fz\x87d4\x14k\x99\xf2\xbbt\xa4\x12:\xa8u\n\x11\x05\xe9\xf9\xa8\xbb&wh[\xa3\x80\x18{G\x12\x0e\x83\xa5\x06]FBB\xca\x06\xa6K\\\xf2V!\xc1\xf5\xe6\xa7G\x1b\xd43\x1d{+0\x1f-\xe0\xbf\xaf\x8a1\x81\xd4\xd8\x01dg\xf9\x04\xbd\xe0\xfc\x1d\xd2\xee+\x84\xdd\xad\xbd\x87\xca\x9d4~\x86\xff\xb8\x8cb,\xcb\x1b\x07\xf9\xe3\x9fr\x88\xf1[5?PK\x03\x04\x14\x00\x00\x00\x08\x00\x00\x98\x9f\xeb\x